In [1]:
# CELL 1: Imports, Setup, and Hyperparameters

# Install necessary libraries if not already installed
# !pip install torch torchvision numpy tqdm matplotlib

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import os
import numpy as np

# Global Settings
NUM_EPOCHS = 100          # was 10
BATCH_SIZE = 128         # was 64
LEARNING_RATE = 0.001     # keep same (good for Adam)
WEIGHT_DECAY = 5e-4       # stronger regularization
DROPOUT_RATE = 0.3        # slightly lower
VALIDATION_SPLIT = 0.1
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CIFAR-10 Statistics
cifar10_mean = [0.4914, 0.4822, 0.4465]
cifar10_std = [0.2023, 0.1994, 0.2010]

# Model Checkpoint Path (used temporarily to save best model during each run)
MODEL_SAVE_PATH = 'best_model_cifar10_temp.pth'

# List to store results from all experiments
RESULTS_TABLE = []

print(f"Running experiments on device: {DEVICE}")
print(f"Number of Epochs set to: {NUM_EPOCHS} (Change to 100 for max accuracy)")

Running experiments on device: cuda
Number of Epochs set to: 100 (Change to 100 for max accuracy)


In [3]:
# CELL 2: Data Preparation and Loading Functions

from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import AutoAugment, AutoAugmentPolicy

def prepare_data(use_augmentation=True):
    # Transformation for Test/Validation data (always consistent)
    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(cifar10_mean, cifar10_std)
    ])
    
    # Training data transformations (with strong augmentation)
    if use_augmentation:
        print("Using Strong Data Augmentation (Crop, Flip, AutoAugment)")
        train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            AutoAugment(policy=AutoAugmentPolicy.CIFAR10),
            transforms.ToTensor(),
            transforms.Normalize(cifar10_mean, cifar10_std)
        ])
    else:
        print("Ablation: NO Data Augmentation used.")
        train_transform = test_transform  # Use simple test transform for training

    # Download and create datasets
    train_full = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

    # Split training data into training and validation sets
    val_size = int(len(train_full) * VALIDATION_SPLIT)
    train_size = len(train_full) - val_size
    train_dataset, val_dataset = random_split(train_full, [train_size, val_size])

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    return train_loader, val_loader, test_loader

print("✅ Data preparation functions defined.")


✅ Data preparation functions defined.


In [5]:
# CELL 3: Model Architecture and Setup Functions

# Your Original SimpleCNN (with configurable Dropout)
class SimpleCNN(nn.Module):
    def __init__(self, dropout_rate=0.0):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 32x32 -> 16x16

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 16x16 -> 8x8

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)   # 8x8 -> 4x4
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 512), # 4096 features
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate), # <- Configurable Dropout
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate), # <- Configurable Dropout
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

def create_resnet18_baseline():
    """Creates the ResNet-18 model for baseline comparison."""
    # Load pre-trained weights from ImageNet
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

    # Replace the final layer for CIFAR-10 (10 classes)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 10)
    
    # Use standard low LR for fine-tuning SOTA model
    return model, 0.0001, 'ResNet-18 (SOTA Baseline)', WEIGHT_DECAY
def create_resnet50_baseline():
    """Creates the ResNet-50 model (stronger SOTA baseline)."""
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 10)
    return model, 0.0001, 'ResNet-50 (SOTA+)', WEIGHT_DECAY

def create_simplecnn_model(ablation_name, dropout_rate=DROPOUT_RATE, weight_decay=WEIGHT_DECAY, learning_rate=LEARNING_RATE):
    """Creates a SimpleCNN instance with specific ablation settings."""
    model = SimpleCNN(dropout_rate=dropout_rate)
    return model, learning_rate, ablation_name, weight_decay

print("Model architectures defined.")

Model architectures defined.


In [7]:
# CELL 4: Training and Evaluation Functions

def validate_model(model, val_loader, criterion):
    """Calculates validation loss and accuracy."""
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)

            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = total_loss / total
    accuracy = 100. * correct / total
    return avg_loss, accuracy


def run_training_experiment(model, lr, name, weight_decay, use_augmentation):
    """Runs a single training experiment with specified settings."""
    global RESULTS_TABLE
    print(f"\n--- Starting Experiment: {name} ---")

    # 1. Data Setup (Ablation Hook for Augmentation)
    train_loader, val_loader, test_loader = prepare_data(use_augmentation=use_augmentation)

    # 2. Setup Model, Loss, Optimizer, and Scheduler
    model = model.to(DEVICE)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # ✅ Added label smoothing
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)  # ✅ Use CosineAnnealingLR

    best_val_accuracy = 0.0

    # 3. Training Loop
    for epoch in range(NUM_EPOCHS):
        model.train()
        train_loss = 0.0

        # Use tqdm for progress bar
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} ({name})", unit="batch")

        for inputs, targets in pbar:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            pbar.set_postfix({'loss': f'{train_loss/len(train_loader):.4f}', 'LR': f'{optimizer.param_groups[0]["lr"]:.6f}'})

        # 4. Validation and Checkpointing
        val_loss, val_acc = validate_model(model, val_loader, criterion)
        scheduler.step()  # ✅ Step cosine scheduler each epoch

        # Save the best model
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), MODEL_SAVE_PATH)

    # 5. Final Test Evaluation (Load Best Model)
    model.load_state_dict(torch.load(MODEL_SAVE_PATH))
    test_loss, test_acc = validate_model(model, test_loader, criterion)

    # Store result for the final table
    RESULTS_TABLE.append({
        'Model': name, 
        'ValidationAcc': best_val_accuracy, 
        'TestAcc': test_acc
    })
    
    print(f"--- Experiment Finished: {name} ---")
    print(f"Final Test Accuracy for {name}: {test_acc:.2f}%")
    
    return best_val_accuracy, test_acc


print("Training and evaluation logic defined.")


Training and evaluation logic defined.


In [9]:
# CELL 5: Master Runner and Final Report Generation

def run_all_experiments():
    """Defines and runs all required experiments sequentially and prints the final report."""
    global RESULTS_TABLE
    RESULTS_TABLE = [] # Reset results

    # --- EXPERIMENT 1: MAIN MODEL (Full SimpleCNN) ---
    # This is your optimized 90.47% model configuration
    model_full, lr_full, name_full, wd_full = create_simplecnn_model("SimpleCNN (Main/Full)")
    run_training_experiment(model_full, lr_full, name_full, wd_full, use_augmentation=True)

    # --- EXPERIMENT 2: SOTA BASELINE (ResNet-50 for High Accuracy) ---
    model_resnet50, lr_resnet50, name_resnet50, wd_resnet50 = create_resnet50_baseline()
    run_training_experiment(model_resnet50, lr_resnet50, name_resnet50, wd_resnet50, use_augmentation=True)

    # --- EXPERIMENT 3: ABLATION 1 (No Data Augmentation) ---
    # Fulfills "Ablation on core modules" requirement
    model_no_aug, lr_no_aug, name_no_aug, wd_no_aug = create_simplecnn_model("Ablation 1 (No Augmentation)")
    run_training_experiment(model_no_aug, lr_no_aug, name_no_aug, wd_no_aug, use_augmentation=False)

    # --- EXPERIMENT 4: ABLATION 2 (No Dropout) ---
    # Fulfills "Ablation on hyper-parameters" requirement
    model_no_dropout, lr_no_dropout, name_no_dropout, wd_no_dropout = create_simplecnn_model("Ablation 2 (No Dropout)", dropout_rate=0.0)
    run_training_experiment(model_no_dropout, lr_no_dropout, name_no_dropout, wd_no_dropout, use_augmentation=True)

    # --- EXPERIMENT 5: ABLATION 3 (No L2 Regularization / Weight Decay) ---
    # Fulfills "Ablation on hyper-parameters" requirement
    model_no_l2, lr_no_l2, name_no_l2, wd_no_l2 = create_simplecnn_model("Ablation 3 (No L2 Decay)", weight_decay=0.0)
    run_training_experiment(model_no_l2, lr_no_l2, name_no_l2, wd_no_l2, use_augmentation=True)

    # --- FINAL REPORT GENERATION ---
    print("\n\n" + "="*70)
    print("FINAL EXPERIMENTAL RESULTS SUMMARY FOR PROJECT REPORT")
    print("="*70)

    # Find the result for the Main Model to use as the baseline for the Ablation table
    main_model_result = next(res for res in RESULTS_TABLE if 'Main' in res['Model'])
    
    # 1. Quantitative Comparison (Baseline) Table
    print("\n\n--- Table 1: Quantitative Comparison (Baseline) ---")
    print("{:<35} {:<15} {:<15}".format("Model", "Test Acc. (%)", "Type"))
    print("-" * 65)
    for res in RESULTS_TABLE:
        model_type = "Custom CNN (Ours)" if 'SimpleCNN' in res['Model'] else "SOTA (ResNet-18)"
        if 'Main' in res['Model'] or 'ResNet-18' in res['Model']:
            print("{:<35} {:<15.2f} {:<15}".format(res['Model'], res['TestAcc'], model_type))


    # 2. Ablation Study Table
    print("\n\n--- Table 2: Ablation Study (3 Groups on SimpleCNN) ---")
    print("{:<35} {:<20} {:<15}".format("Experiment", "Configuration", "Test Acc. (%)"))
    print("-" * 70)
    
    # Print Main Model first for context
    print("{:<35} {:<20} {:<15.2f}".format(
        "Main Model (Best)",
        "Full Features",
        main_model_result['TestAcc']
    ))
    
    # Print Ablations
    for res in RESULTS_TABLE:
        if 'Ablation' in res['Model']:
            config = res['Model'].split('(')[-1].replace(')', '')
            print("{:<35} {:<20} {:<15.2f}".format(
                res['Model'],
                f"No {config}",
                res['TestAcc']
            ))


    print("\nAll required experiments are complete.")
    
    # Clean up temporary saved model file
    if os.path.exists(MODEL_SAVE_PATH):
        os.remove(MODEL_SAVE_PATH)

# Execute the master function to start the entire process
if __name__ == '__main__':
    run_all_experiments()

print("Master execution function defined. Run this cell to start all 5 experiments.")


--- Starting Experiment: SimpleCNN (Main/Full) ---
Using Strong Data Augmentation (Crop, Flip, AutoAugment)


Epoch 100/100 (SimpleCNN (Main/Full)): 100%|██████████| 352/352 [00:16<00:00, 21.26batch/s, loss=0.8093, LR=0.000000]


--- Experiment Finished: SimpleCNN (Main/Full) ---
Final Test Accuracy for SimpleCNN (Main/Full): 90.31%

--- Starting Experiment: ResNet-50 (SOTA+) ---
Using Strong Data Augmentation (Crop, Flip, AutoAugment)


Epoch 100/100 (ResNet-50 (SOTA+)): 100%|██████████| 352/352 [00:17<00:00, 19.90batch/s, loss=0.6292, LR=0.000000]


--- Experiment Finished: ResNet-50 (SOTA+) ---
Final Test Accuracy for ResNet-50 (SOTA+): 90.66%

--- Starting Experiment: Ablation 1 (No Augmentation) ---
Ablation: NO Data Augmentation used.


...

--- Experiment Finished: Ablation 1 (No Augmentation) ---
Final Test Accuracy for Ablation 1 (No Augmentation): 80.81%

--- Starting Experiment: Ablation 2 (No Dropout) ---
Using Strong Data Augmentation (Crop, Flip, AutoAugment)


Epoch 100/100 (Ablation 2 (No Dropout)): 100%|██████████| 352/352 [00:17<00:00, 19.86batch/s, loss=0.7854, LR=0.000000]


--- Experiment Finished: Ablation 2 (No Dropout) ---
Final Test Accuracy for Ablation 2 (No Dropout): 90.13%

--- Starting Experiment: Ablation 3 (No L2 Decay) ---
Using Strong Data Augmentation (Crop, Flip, AutoAugment)


Epoch 100/100 (Ablation 3 (No L2 Decay)): 100%|██████████| 352/352 [00:16<00:00, 21.62batch/s, loss=0.7980, LR=0.000000]


--- Experiment Finished: Ablation 3 (No L2 Decay) ---
Final Test Accuracy for Ablation 3 (No L2 Decay): 90.33%


FINAL EXPERIMENTAL RESULTS SUMMARY FOR PROJECT REPORT


--- Table 1: Quantitative Comparison (Baseline) ---
Model                               Test Acc. (%)   Type           
-----------------------------------------------------------------
SimpleCNN (Main/Full)               90.31           Custom CNN (Ours)


--- Table 2: Ablation Study (3 Groups on SimpleCNN) ---
Experiment                          Configuration        Test Acc. (%)  
----------------------------------------------------------------------
Main Model (Best)                   Full Features        90.31          
Ablation 1 (No Augmentation)        No No Augmentation   80.81          
Ablation 2 (No Dropout)             No No Dropout        90.13          
Ablation 3 (No L2 Decay)            No No L2 Decay       90.33          

All required experiments are complete.
Master execution function defined. Run th